In [1]:
import pandas as pd
import numpy as np
import re
import openpyxl
import easygui
import os

In [2]:
#仓库销售编码字典
fname0 =  r"F:\a00nutstore\008\zw08\用友报价\仓库销售编码字典.xlsx"
df0 = pd.read_excel(fname0)
dic_ck_xs = dict(zip(df0['仓库货号'],df0['销售货号']))
dic_xs_ck = {v:k for k,v in dic_ck_xs.items()}


In [3]:
#产成品仓库库存
def chuliChangku(fname,sheet_name):
    df = pd.read_excel(fname,sheet_name = sheet_name)
    #删除第一列的空行
    df = df[~df['类别'].isna()]
    df = df[~df['类别'].str.contains('小计|累计|合   计')]
    df = df[~(abs(df['本日数量'] - 0) < 0.00001)]
    return df
fname = r"F:\a00nutstore\008\zw08\2024\2024-06\2024.6.25数据.xls"
sheet_name  =  r'6.25'
df = chuliChangku(fname,sheet_name)
df




,类别,货号,品名,含量,单位,上日,本日入库,本日出库,结存,上日数量,入库数量,出库数量,本日数量
0,A16K黑卡缝线纯净版,EFBA460HB,16K60型黑卡缝线空白本,200.0,本,30.0,0.0,0.0,30.0,6000,0,0,6000
1,A16K黑卡缝线纯净版,EFL1660HB,16K60型黑卡缝线横线本,200.0,本,49.0,0.0,0.0,49.0,9800,0,0,9800
2,A16K黑卡缝线纯净版,EFLA460HB,A4-60型黑卡缝线横线本,200.0,本,36.0,0.0,0.0,36.0,7200,0,0,7200
3,A16K黑卡缝线纯净版,EFY1660HB,16K60型黑卡缝线英语本,200.0,本,1.0,0.0,0.0,1.0,200,0,0,200
4,A16K黑卡缝线纯净版,EFYA460HB,A4-60型黑卡缝线英语本,200.0,本,6.0,0.0,0.0,6.0,1200,0,0,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,Z废次品,N0821废次品,N0821废次品,360.0,本,37.0,0.0,0.0,37.0,13320,0,0,13320
1678,Z废次品,N0822废次品,N0822废次品,280.0,本,34.0,0.0,0.0,34.0,9520,0,0,9520
1679,Z废次品,N0823废次品,N0823废次品,240.0,本,27.0,0.0,0.0,27.0,6480,0,0,6480
1680,Z废次品,N0824废次品,N0824废次品,160.0,本,10.0,0.0,0.0,10.0,1600,0,0,1600


In [4]:
def addXiaoshouhuohao(df,dic_ck_xs):
    df_ck = df.assign(销售货号 = df['货号'].map(dic_ck_xs))
    gp = df_ck.groupby('货号')
    df_ck = pd.DataFrame(gp.sum())
    df_ck = df_ck[['销售货号','本日数量']]
    df_ck = df_ck.reset_index()
    return df_ck
df_ck = addXiaoshouhuohao(df,dic_ck_xs)
df_ck = df_ck[['本日数量','销售货号','货号']]
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck.to_excel('仓库库存.xlsx',index = False)

df_ck

C:\Users\redda\AppData\Local\Temp\ipykernel_26184\3041926149.py:11: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_ck.to_excel('仓库库存.xlsx',index = False)


,本日数量,销售货号,仓库货号
0,2580,1101,1101
1,1080,1103,1103
2,1260,1105,1105
3,4980,1106,1106
4,1140,1107,1107
...,...,...,...
1537,2240,瑞典A460g线条,瑞典A460g线条
1538,40,瑞典A470g线条,瑞典A470g线条
1539,252,美国A6铁订本*,美国A6铁订本(枫叶)*
1540,324,美国A6铁订本(枫叶)*,美国A6铁订本*


In [8]:
#销售编码
def xiaoshou(fname_xs):
    df_xs = pd.read_excel(fname_xs)
    df_xs = df_xs[~df_xs['类别'].isna()]
    return df_xs

        
fname_xs = r"F:\a00nutstore\008\zw08\用友报价\销售仓库价格库存1.xlsx" 
df_xs = xiaoshou(fname_xs)
xs_lst = [
 '类别',
 '货号',
 '品名',
 '销售部存货大类编码',
 '含量',
 '汉办',
 '汉口北电商',
 '外地',
]
df_xs = df_xs[xs_lst ]
df_xs


,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地
0,多联单据,5007,48K75P送货单(三联、多栏),01010303,360,1.4800,1.4800,1.4800
1,多联单据,5008,48K75P缴款单(三联、多栏),01010303,360,1.4800,1.4800,1.4800
2,多联单据,5009,48K75P收据(三联、单栏),01010303,360,1.4800,1.4800,1.4800
3,多联单据,5014,40K75P菜单(三联、竖式多栏),01010303,360,1.4800,1.4800,1.4800
4,多联单据,5017,32K75P送货单(三联、竖式多栏),01010303,240,1.9300,1.9300,1.9300
...,...,...,...,...,...,...,...,...
1507,外贸订制品,瑞典A460g方格,瑞典A460g方格,010214,40,0.0000,0.0000,0.0000
1508,外贸订制品,瑞典A460g线条,瑞典A460g线条,010214,40,0.0000,0.0000,0.0000
1509,外贸订制品,瑞典A470g线条,瑞典A470g线条,010214,40,0.0000,0.0000,0.0000
1510,装订配件,商品卡,48K商品卡,010106,5400,0.1100,0.1100,0.1100


In [ ]:
#新增仓库货号
lst = set(df_ck['仓库货号'])-set(df_xs['仓库货号'])
lst


In [ ]:
df_ck_diff = pd.DataFrame({'仓库货号':list(lst)})
df_ck_diff

In [10]:
#销售货号唯一化处理
def weiyihuXiaoshouhuohao(df_xs,df_ck,dic_xs_ck):
    merge = pd.merge(df_xs,df_ck,left_on = '货号',right_on = '销售货号',how = 'right')
    merge['本日数量本'] = merge['本日数量'] 
    merge['本日数量件'] = merge['本日数量'] /merge['含量']
    del merge['本日数量']
    del merge['销售货号']
    gp = merge.groupby('货号')
    dic_agg = {'含量':'mean','汉办':'mean','汉口北电商':'mean','外地':'mean','本日数量本':'sum'}
    right = gp.agg(dic_agg)
    right = right.reset_index()
    left = df_xs[[
 '类别',
 '货号',
 '品名',
 '销售部存货大类编码'
]]
    mg  = pd.merge(left,right,left_on = '货号',right_on = '货号')
    mg['本日数量件'] = mg['本日数量本']/mg['含量']
    mg = mg.assign(仓库货号 = mg['货号'].map(dic_xs_ck))
    return mg

result = weiyihuXiaoshouhuohao(df_xs,df_ck,dic_xs_ck)
result 
    
        

,类别,货号,品名,销售部存货大类编码,含量,汉办,汉口北电商,外地,本日数量本,本日数量件,仓库货号
0,多联单据,5007,48K75P送货单(三联、多栏),01010303,360.0,1.4800,1.4800,1.4800,35640,99.0,5007A
1,多联单据,5008,48K75P缴款单(三联、多栏),01010303,360.0,1.4800,1.4800,1.4800,10800,30.0,5008A
2,多联单据,5009,48K75P收据(三联、单栏),01010303,360.0,1.4800,1.4800,1.4800,151920,422.0,5009A
3,多联单据,5014,40K75P菜单(三联、竖式多栏),01010303,360.0,1.4800,1.4800,1.4800,24840,69.0,5014A
4,多联单据,5017,32K75P送货单(三联、竖式多栏),01010303,240.0,1.9300,1.9300,1.9300,15840,66.0,5017A
...,...,...,...,...,...,...,...,...,...,...,...
1507,外贸订制品,瑞典A460g方格,瑞典A460g方格,010214,40.0,0.0000,0.0000,0.0000,720,18.0,瑞典A460g方格
1508,外贸订制品,瑞典A460g线条,瑞典A460g线条,010214,40.0,0.0000,0.0000,0.0000,2240,56.0,瑞典A460g线条
1509,外贸订制品,瑞典A470g线条,瑞典A470g线条,010214,40.0,0.0000,0.0000,0.0000,40,1.0,瑞典A470g线条
1510,装订配件,商品卡,48K商品卡,010106,5400.0,0.1100,0.1100,0.1100,113400,21.0,商品卡


In [12]:
result.to_excel('销售仓库价格库存1.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_26184\1983119061.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  result.to_excel('销售仓库价格库存1.xlsx',index = False)


In [36]:
df_ck0 = chuliChangku(r"F:\a00nutstore\008\zw08\用友报价\仓库产成品5.26-6.12出入库资料.xls",sheet_name = 'AAA')
df_ck0

,类别,货号,品名,含量,单位,上日,本日入库,本日出库,结存,上日数量,入库数量,出库数量,本日数量
0,A16K黑卡缝线纯净版,EFBA460HB,16K60型黑卡缝线空白本,200.0,本,30.0,0.0,0.0,30.0,6000,0,0,6000
1,A16K黑卡缝线纯净版,EFL1660HB,16K60型黑卡缝线横线本,200.0,本,72.0,0.0,10.0,62.0,14400,0,2000,12400
2,A16K黑卡缝线纯净版,EFLA460HB,A4-60型黑卡缝线横线本,200.0,本,36.0,0.0,0.0,36.0,7200,0,0,7200
3,A16K黑卡缝线纯净版,EFY1660HB,16K60型黑卡缝线英语本,200.0,本,1.0,0.0,0.0,1.0,200,0,0,200
4,A16K黑卡缝线纯净版,EFYA460HB,A4-60型黑卡缝线英语本,200.0,本,6.0,0.0,0.0,6.0,1200,0,0,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,Z废次品,N0821废次品,N0821废次品,360.0,本,28.0,8.0,0.0,36.0,10080,2880,0,12960
1640,Z废次品,N0822废次品,N0822废次品,280.0,本,34.0,0.0,0.0,34.0,9520,0,0,9520
1641,Z废次品,N0823废次品,N0823废次品,240.0,本,27.0,0.0,0.0,27.0,6480,0,0,6480
1642,Z废次品,N0824废次品,N0824废次品,160.0,本,10.0,0.0,0.0,10.0,1600,0,0,1600


In [14]:
df_ck0 = df_ck0[['本日数量','销售货号','货号']]
df_ck0 = df_ck0.rename(columns = {'货号':'仓库货号'})
df_ck0

KeyError: "['销售货号'] not in index"

In [15]:
df_ck0

,类别,货号,品名,含量,单位,上日,本日入库,本日出库,结存,上日数量,入库数量,出库数量,本日数量
0,A16K黑卡缝线纯净版,EFBA460HB,16K60型黑卡缝线空白本,200.0,本,30.0,0.0,0.0,30.0,6000,0,0,6000
1,A16K黑卡缝线纯净版,EFL1660HB,16K60型黑卡缝线横线本,200.0,本,72.0,0.0,10.0,62.0,14400,0,2000,12400
2,A16K黑卡缝线纯净版,EFLA460HB,A4-60型黑卡缝线横线本,200.0,本,36.0,0.0,0.0,36.0,7200,0,0,7200
3,A16K黑卡缝线纯净版,EFY1660HB,16K60型黑卡缝线英语本,200.0,本,1.0,0.0,0.0,1.0,200,0,0,200
4,A16K黑卡缝线纯净版,EFYA460HB,A4-60型黑卡缝线英语本,200.0,本,6.0,0.0,0.0,6.0,1200,0,0,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,Z废次品,N0821废次品,N0821废次品,360.0,本,28.0,8.0,0.0,36.0,10080,2880,0,12960
1640,Z废次品,N0822废次品,N0822废次品,280.0,本,34.0,0.0,0.0,34.0,9520,0,0,9520
1641,Z废次品,N0823废次品,N0823废次品,240.0,本,27.0,0.0,0.0,27.0,6480,0,0,6480
1642,Z废次品,N0824废次品,N0824废次品,160.0,本,10.0,0.0,0.0,10.0,1600,0,0,1600


In [16]:
dic_ck_xs

{'1101': '1101',
 '1103': '1103',
 '1105': '1105',
 '1106': '1106',
 '1107': '1107',
 '1109': '1109',
 '1110': '1110',
 '1113': '1113',
 '1115': '1115',
 '1116': '1116',
 '1117': '1117',
 '1120': '1120',
 '1121': '1121',
 '1122': '1122',
 '1123': '1123',
 '1124': '1124',
 '1125': '1125',
 '1126': '1126',
 '1206': '1206',
 '1208': '1208',
 '1210': '1210',
 '1225': '1225',
 '1508废次品': '1508废次品',
 '1601': '1601',
 '1602': '1602',
 '1603': '1603',
 '16K30牛皮草稿本湖南': '16K30页牛皮草稿本湖南',
 '16K30牛皮分区草稿本': '16K30页牛皮草稿本湖南',
 '16K50牛皮草稿本湖南': '16K50页牛皮草稿本湖南',
 '16KPP账夹': '16KPP账夹',
 '16K备课本-30P': '16K备课本-30P',
 '16K备课本-50Pa': '16K备课本-50Pa',
 '16K废次品': '16K废次品',
 '2001': '2001',
 '2002': '2002',
 '2003': '2003',
 '2004': '2004',
 '2005': '2005',
 '2301': '2301',
 '24K大横格本(贵州)': '24K大横格本(贵州)',
 '24K单行本(贵州)': '24K单行本（贵州）a',
 '24K课文簿(贵州)': '24K课文簿(贵州)',
 '24K拼音写字簿(贵州)': '24K拼音写字簿(贵州)',
 '24K数学本(贵州)': '24K数学本(贵州)',
 '24K中方格簿(贵州)': '24K中方格簿(贵州)',
 '25KPP账夹(红)': '25KPP账夹',
 '3001': '3001',
 '3002': '3002',
 

In [37]:
df_ck0 = addXiaoshouhuohao(df_ck0,dic_ck_xs)
df_ck0

,货号,销售货号,本日数量
0,1052399,0,39048
1,1052400,0,39864
2,1052402,0,39768
3,1052403.,0,39504
4,1101,1101,2640
...,...,...,...
1441,瑞典A460g线条,瑞典A460g线条,2240
1442,瑞典A470g线条,瑞典A470g线条,40
1443,美国A6铁订本(枫叶)*,美国A6铁订本*,252
1444,美国A6铁订本*,美国A6铁订本(枫叶)*,324


In [29]:
def cangkuDiff(df0,df1):
    df0 = df0.rename(columns = {'货号':'仓库货号'})
    df1 = df1.rename(columns = {'货号':'仓库货号'})
    df0 = df0[['货号','本日数量']]
    df1 = df1[['货号','本日数量']]
    merge = pd.merge(df0,df1,left_on= '仓库货号',right_on= '仓库货号',how = 'outer')
    return merge
diff = cangkuDiff(df_ck0,df_ck)
diff
    
    

KeyError: "['货号'] not in index"

In [19]:
df_ck0

,货号,销售货号,本日数量
0,1052399,0,39048
1,1052400,0,39864
2,1052402,0,39768
3,1052403.,0,39504
4,1101,1101,2640
...,...,...,...
1441,瑞典A460g线条,瑞典A460g线条,2240
1442,瑞典A470g线条,瑞典A470g线条,40
1443,美国A6铁订本(枫叶)*,美国A6铁订本*,252
1444,美国A6铁订本*,美国A6铁订本(枫叶)*,324


In [23]:
df_ck

,本日数量,销售货号,仓库货号
0,2580,1101,1101
1,1080,1103,1103
2,1260,1105,1105
3,4980,1106,1106
4,1140,1107,1107
...,...,...,...
1537,2240,瑞典A460g线条,瑞典A460g线条
1538,40,瑞典A470g线条,瑞典A470g线条
1539,252,美国A6铁订本*,美国A6铁订本(枫叶)*
1540,324,美国A6铁订本(枫叶)*,美国A6铁订本*


In [38]:
df_ck0 = df_ck0.rename(columns = {'货号':'仓库货号'})
df_ck0

,仓库货号,销售货号,本日数量
0,1052399,0,39048
1,1052400,0,39864
2,1052402,0,39768
3,1052403.,0,39504
4,1101,1101,2640
...,...,...,...
1441,瑞典A460g线条,瑞典A460g线条,2240
1442,瑞典A470g线条,瑞典A470g线条,40
1443,美国A6铁订本(枫叶)*,美国A6铁订本*,252
1444,美国A6铁订本*,美国A6铁订本(枫叶)*,324


In [39]:
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck

,本日数量,销售货号,仓库货号
0,2580,1101,1101
1,1080,1103,1103
2,1260,1105,1105
3,4980,1106,1106
4,1140,1107,1107
...,...,...,...
1537,2240,瑞典A460g线条,瑞典A460g线条
1538,40,瑞典A470g线条,瑞典A470g线条
1539,252,美国A6铁订本*,美国A6铁订本(枫叶)*
1540,324,美国A6铁订本(枫叶)*,美国A6铁订本*


In [46]:
merge = pd.merge(df_ck0,df_ck,left_on= '仓库货号',right_on= '仓库货号',how = 'outer')
merge

,仓库货号,销售货号_x,本日数量_x,本日数量_y,销售货号_y
0,1052399,0,39048.0,NaN,NaN
1,1052400,0,39864.0,NaN,NaN
2,1052402,0,39768.0,NaN,NaN
3,1052403.,0,39504.0,NaN,NaN
4,1101,1101,2640.0,2580.0,1101
...,...,...,...,...,...
1562,瑞典A460g线条,瑞典A460g线条,2240.0,2240.0,瑞典A460g线条
1563,瑞典A470g线条,瑞典A470g线条,40.0,40.0,瑞典A470g线条
1564,美国A6铁订本(枫叶)*,美国A6铁订本*,252.0,252.0,美国A6铁订本*
1565,美国A6铁订本*,美国A6铁订本(枫叶)*,324.0,324.0,美国A6铁订本(枫叶)*


In [47]:
merge['本日数量_y'] = merge['本日数量_y'].fillna(0)
merge['本日数量_x'] = merge['本日数量_x'].fillna(0)
merge

,仓库货号,销售货号_x,本日数量_x,本日数量_y,销售货号_y
0,1052399,0,39048.0,0.0,NaN
1,1052400,0,39864.0,0.0,NaN
2,1052402,0,39768.0,0.0,NaN
3,1052403.,0,39504.0,0.0,NaN
4,1101,1101,2640.0,2580.0,1101
...,...,...,...,...,...
1562,瑞典A460g线条,瑞典A460g线条,2240.0,2240.0,瑞典A460g线条
1563,瑞典A470g线条,瑞典A470g线条,40.0,40.0,瑞典A470g线条
1564,美国A6铁订本(枫叶)*,美国A6铁订本*,252.0,252.0,美国A6铁订本*
1565,美国A6铁订本*,美国A6铁订本(枫叶)*,324.0,324.0,美国A6铁订本(枫叶)*


In [48]:
merge['diff'] = merge['本日数量_y']- merge['本日数量_x']
merge

,仓库货号,销售货号_x,本日数量_x,本日数量_y,销售货号_y,diff
0,1052399,0,39048.0,0.0,NaN,-39048.0
1,1052400,0,39864.0,0.0,NaN,-39864.0
2,1052402,0,39768.0,0.0,NaN,-39768.0
3,1052403.,0,39504.0,0.0,NaN,-39504.0
4,1101,1101,2640.0,2580.0,1101,-60.0
...,...,...,...,...,...,...
1562,瑞典A460g线条,瑞典A460g线条,2240.0,2240.0,瑞典A460g线条,0.0
1563,瑞典A470g线条,瑞典A470g线条,40.0,40.0,瑞典A470g线条,0.0
1564,美国A6铁订本(枫叶)*,美国A6铁订本*,252.0,252.0,美国A6铁订本*,0.0
1565,美国A6铁订本*,美国A6铁订本(枫叶)*,324.0,324.0,美国A6铁订本(枫叶)*,0.0


In [49]:
merge.to_excel('9999.xlsx',index = False)

C:\Users\redda\AppData\Local\Temp\ipykernel_26184\1875209939.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merge.to_excel('9999.xlsx',index = False)


In [50]:
merge['销售货号_x'] =  merge['仓库货号'].map(dic_ck_xs)
merge

,仓库货号,销售货号_x,本日数量_x,本日数量_y,销售货号_y,diff
0,1052399,NaN,39048.0,0.0,NaN,-39048.0
1,1052400,NaN,39864.0,0.0,NaN,-39864.0
2,1052402,NaN,39768.0,0.0,NaN,-39768.0
3,1052403.,NaN,39504.0,0.0,NaN,-39504.0
4,1101,1101,2640.0,2580.0,1101,-60.0
...,...,...,...,...,...,...
1562,瑞典A460g线条,瑞典A460g线条,2240.0,2240.0,瑞典A460g线条,0.0
1563,瑞典A470g线条,瑞典A470g线条,40.0,40.0,瑞典A470g线条,0.0
1564,美国A6铁订本(枫叶)*,美国A6铁订本*,252.0,252.0,美国A6铁订本*,0.0
1565,美国A6铁订本*,美国A6铁订本(枫叶)*,324.0,324.0,美国A6铁订本(枫叶)*,0.0


In [51]:
del merge['销售货号_y']
merge = merge.rename({'销售货号_x':'销售货号'})
merge.to_excel('9999.xlsx',index = False)


C:\Users\redda\AppData\Local\Temp\ipykernel_26184\323739650.py:3: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  merge.to_excel('9999.xlsx',index = False)
